# Machine learning
Escolhi "Previsão de vendas"  

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import joblib  

In [9]:
df = pd.read_csv("./dataset/superstore.csv", encoding='ISO-8859-1')

### Removendo outliers

In [11]:
# N de linhas antes de tirar os outliers
len(df)

9994

In [12]:
df = df[df['Sales'] <= 6000]

df = df[df['Quantity'] <= 9]

df = df[(df['Profit'] >= -2000) & (df['Profit'] <= 3900)]

print("Número de linhas após remoção de outliers:", len(df))

Número de linhas após remoção de outliers: 9804


In [13]:
# Resetar o índice para que fique sequencial 
df = df.reset_index(drop=True)

In [14]:
# Utilizando apenas colunas nescessárias para esta análise
numerical_features = ['Sales', 'Segment', 'Quantity', 'Profit']
categorical_features = ['Ship Mode', 'Category', 'Sub-Category', 'Region']

In [15]:
# Atribuindo apenas as colunas que vou utilizar ao dataframe
df = df[numerical_features + categorical_features ]

In [16]:
df.head(2)

,Sales,Segment,Quantity,Profit,Ship Mode,Category,Sub-Category,Region
0,261.96,Consumer,2,41.9136,Second Class,Furniture,Bookcases,South
1,731.94,Consumer,3,219.5820,Second Class,Furniture,Chairs,South


### Tratando a coluna "Ship Mode"
Vou usar o "ordinal encoder" pois esta coluna tem relevancia entre elas, cada classe vai melhorando a rapidez do envio.

In [17]:
# Vamos tratar a coluna, para nosso modelo poder entender
df['Ship Mode'].value_counts()

Ship Mode
Standard Class    5848
Second Class      1906
First Class       1514
Same Day           536
Name: count, dtype: int64

In [18]:
# Criando o objeto 
oe = OrdinalEncoder(categories=[['Standard Class','Second Class','First Class','Same Day']], # escolhendo a ordem, ele começa com 0
                    handle_unknown='use_encoded_value', # como tratar os valores vazios
                    unknown_value=-1, # valor atribuido aos valores vazios
                    dtype='int32' # Escolhendo o tipo do número, pois se vc n escolher inteiro ele vai vir como float
                   )

In [19]:
# Fazendo o tratamento
oe = oe.fit(df[['Ship Mode']])
oe.transform(df[['Ship Mode']])

# Criando um DF
oe_df = pd.DataFrame(oe.transform(df[['Ship Mode']]),columns=['Ship Mode oe'])

# Unindo com o dataset original e removendo a coluna antiga
df = pd.concat([df.drop(columns='Ship Mode'),oe_df],axis=1)

In [20]:
df.head(2)

,Sales,Segment,Quantity,Profit,Category,Sub-Category,Region,Ship Mode oe
0,261.96,Consumer,2,41.9136,Furniture,Bookcases,South,1
1,731.94,Consumer,3,219.5820,Furniture,Chairs,South,1


### Tratando a coluna  "Segment", "Category", "Sub_Category", "Region"
Vou usar o one hot pois aqui elas não tem relevancia entre si

In [21]:
# Lista de colunas categóricas a serem codificadas
colunas_oh = ['Segment', 'Category', 'Sub-Category', 'Region']

In [22]:
from sklearn.preprocessing import OneHotEncoder
# Inicializar o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# Ajustar e transformar os dados
encoded_data = encoder.fit_transform(df[colunas_oh])
encoded_columns = encoder.get_feature_names_out(colunas_oh)

In [23]:
# Criar DataFrame com os dados transformados
encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns, index=df.index)
# Concatenar com o DataFrame original (removendo as colunas originais)
df = pd.concat([df.drop(columns=colunas_oh), encoded_df], axis=1)

In [24]:
df.head(3)

,Sales,Quantity,Profit,Ship Mode oe,Segment_Consumer,Segment_Corporate,Segment_Home Office,Category_Furniture,Category_Office Supplies,Category_Technology,...,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables,Region_Central,Region_East,Region_South,Region_West
0,261.96,2,41.9136,1,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,731.94,3,219.5820,1,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,14.62,2,6.8714,1,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Começando o modelo

In [25]:
# Dividir dados em treino e teste
X = df.drop('Sales', axis=1)
y = df['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Escolhendo os modelos
modelos = {
    'RandomForest': {
         'estimator': RandomForestRegressor(random_state=42),
         'param_grid': {
              'model__n_estimators': [50, 100, 200],
              'model__max_depth': [None, 10, 20],
              'model__min_samples_split': [2, 5, 10]
         }
    },
    'GradientBoosting': {
         'estimator': GradientBoostingRegressor(random_state=42),
         'param_grid': {
              'model__n_estimators': [50, 100, 200],
              'model__learning_rate': [0.01, 0.1, 0.2],
              'model__max_depth': [3, 5, 7]
         }
    },
    'Ridge': {
         'estimator': Ridge(),
         'param_grid': {
              'model__alpha': [0.1, 1.0, 10.0]
         }
    },
    'SVR': {
         'estimator': SVR(),
         'param_grid': {
              'model__C': [0.1, 1, 10],
              'model__epsilon': [0.1, 0.2, 0.5],
              'model__kernel': ['rbf']  # Se preferir, pode testar também 'linear'
         }
    }
}

In [27]:
# Dicionário para armazenar os resultados
resultados = {}

# Loop para treinar cada modelo 
for nome, config in modelos.items():
    print(f"\nTreinando modelo: {nome}")
    
   
    pipeline = Pipeline([
         ('scaler', StandardScaler()),
         ('model', config['estimator'])
    ])
    
    
    grid_search = GridSearchCV(
         estimator=pipeline,
         param_grid=config['param_grid'],
         cv=5,
         scoring='neg_mean_squared_error',  # MSE negativo (quanto maior, melhor)
         n_jobs=-1,
         verbose=1
    )
    
    
    grid_search.fit(X_train, y_train)
    
   
    melhor_modelo = grid_search.best_estimator_
    y_pred = melhor_modelo.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    resultados[nome] = {
         'melhores_parametros': grid_search.best_params_,
         'score_cv': grid_search.best_score_,
         'mse_teste': mse,
         'r2_teste': r2
    }
    
    print(f"Melhores parâmetros para {nome}: {grid_search.best_params_}")
    print(f"Score de treino (neg MSE): {grid_search.best_score_:.4f}")
    print(f"MSE no teste: {mse:.4f}")
    print(f"R² no teste: {r2:.4f}")
    print("-"*50)



Treinando modelo: RandomForest
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Melhores parâmetros para RandomForest: {'model__max_depth': None, 'model__min_samples_split': 5, 'model__n_estimators': 200}
Score de treino (neg MSE): -37752.7144
MSE no teste: 24907.3103
R² no teste: 0.8197
--------------------------------------------------

Treinando modelo: GradientBoosting
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Melhores parâmetros para GradientBoosting: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 200}
Score de treino (neg MSE): -33207.3808
MSE no teste: 24574.0524
R² no teste: 0.8221
--------------------------------------------------

Treinando modelo: Ridge
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Melhores parâmetros para Ridge: {'model__alpha': 10.0}
Score de treino (neg MSE): -101753.0596
MSE no teste: 62927.8674
R² no teste: 0.5445
--------------------------------------------------

Treinando mo

In [28]:
# Exibindo um resumo final dos resultados:
print("\nResumo dos resultados:")
for nome, res in resultados.items():
    print(f"{nome}: R² no teste = {res['r2_teste']:.4f} | MSE no teste = {res['mse_teste']:.4f}")


Resumo dos resultados:
RandomForest: R² no teste = 0.8197 | MSE no teste = 24907.3103
GradientBoosting: R² no teste = 0.8221 | MSE no teste = 24574.0524
Ridge: R² no teste = 0.5445 | MSE no teste = 62927.8674
SVR: R² no teste = 0.3337 | MSE no teste = 92054.1113


## Conclusão
O modelo Gradient Boosting apresentou o melhor desempenho, com o maior R² (0.8221) e o menor MSE (24.574), indicando que explica melhor a variação dos dados e com menor erro. 

O Random Forest teve um desempenho muito próximo, também sendo uma boa escolha.  

Já o Ridge e o SVR tiveram desempenhos significativamente piores, com R² baixos e erros elevados, sugerindo que não são adequados para esse problema.    

**Assim, o Gradient Boosting é a melhor opção com 82% de acertividade na métrica R2 e menor MSE 24.574.**

### Abaixo é opicional
Salvando o modelo, já vou deixar o modelo salvo na pasta.

In [31]:
melhor_modelo = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GradientBoostingRegressor(
        learning_rate=0.1,
        max_depth=5,
        n_estimators=200,
        random_state=42
    ))
])

# Treinar o modelo com os dados de treino
melhor_modelo.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 GradientBoostingRegressor(max_depth=5, n_estimators=200,
                                           random_state=42))])

In [32]:
# Salvar o modelo treinado
joblib.dump(melhor_modelo, 'melhor_modelo.pkl')
print("Modelo GradientBoosting salvo como 'melhor_modelo.pkl'")

Modelo GradientBoosting salvo como 'melhor_modelo.pkl'
